In [11]:
import requests
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, Owlv2ForObjectDetection
from transformers.utils.constants import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD

In [2]:
import jsonlines
import torchaudio
from datasets import Dataset
from pathlib import Path
import torch, random
import librosa, os
import IPython.display as ipd
from dotenv import load_dotenv
import evaluate
from spellchecker import SpellChecker
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_metric
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
import torch
import cv2
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [12]:
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
# input_dir = Path(f"../../data/{TEAM_TRACK}/train")
results_dir = Path(f"/home/jupyter/{TEAM_NAME}")
# results_dir = Path("results")
results_dir.mkdir(parents=True, exist_ok=True)
data = {'image': [], 'annotations': []}
count = 0

with jsonlines.open(input_dir / "vlm.jsonl") as reader:
    for obj in reader:
        if count < 2:  # Check if we have processed less than 10 entries
            data['image'].append(obj['image'])
            data['annotations'].append(obj['annotations'])
            count += 1
        else:
            break  

In [13]:
data

{'image': ['image_0.jpg', 'image_1.jpg'],
 'annotations': [[{'caption': 'grey missile', 'bbox': [912, 164, 48, 152]},
   {'caption': 'red, white, and blue light aircraft',
    'bbox': [1032, 80, 24, 28]},
   {'caption': 'green and black missile', 'bbox': [704, 508, 76, 64]},
   {'caption': 'white and red helicopter', 'bbox': [524, 116, 112, 48]}],
  [{'caption': 'grey camouflage fighter jet', 'bbox': [1112, 172, 64, 36]},
   {'caption': 'grey and white fighter plane', 'bbox': [1108, 512, 144, 48]},
   {'caption': 'white and black drone', 'bbox': [356, 452, 48, 32]},
   {'caption': 'white and black fighter jet', 'bbox': [404, 156, 48, 36]},
   {'caption': 'white missile', 'bbox': [544, 112, 40, 40]},
   {'caption': 'black and white commercial aircraft',
    'bbox': [808, 504, 68, 68]}]]}

In [7]:
# import cv2
# import torch
# from transformers import AutoProcessor, Owlv2ForObjectDetection
# from PIL import Image
# import numpy as np

# # Load the model and processor
# processor = AutoProcessor.from_pretrained("google/owlv2-base-patch16-ensemble")
# model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble")

# # Mean and standard deviation for normalization
# OPENAI_CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
# OPENAI_CLIP_STD = (0.26862954, 0.26130258, 0.27577711)

# # Function to preprocess the image
# def get_preprocessed_image(pixel_values):
#     pixel_values = pixel_values.squeeze().numpy()
#     unnormalized_image = (pixel_values * np.array(OPENAI_CLIP_STD)[:, None, None]) + np.array(OPENAI_CLIP_MEAN)[:, None, None]
#     unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
#     unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
#     unnormalized_image = Image.fromarray(unnormalized_image)
#     return unnormalized_image

# # Assuming 'data' is a list or array containing images
# for image_tensor in data[]:
#     # Convert tensor to NumPy array
#     image_np = image_tensor.permute(1, 2, 0).numpy()

#     # Convert to RGB
#     image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

#     # Convert to PIL image
#     image_pil = Image.fromarray(image_rgb)

#     # Preprocess the image
#     inputs = processor(images=image_pil, return_tensors="pt")

#     # Forward pass
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Note: boxes need to be visualized on the padded, unnormalized image
#     # hence we'll set the target image sizes (height, width) based on that

#     unnormalized_image = get_preprocessed_image(inputs.pixel_values)

#     target_sizes = torch.Tensor([unnormalized_image.size[::-1]])
#     # Convert outputs (bounding boxes and class logits) to final bounding boxes and scores
#     results = processor.post_process_object_detection(
#         outputs=outputs, threshold=0.2, target_sizes=target_sizes
#     )

#     boxes, scores, labels = results[0]["boxes"], results[0]["scores"], results[0]["labels"]

#     for box, score, label in zip(boxes, scores, labels):
#         box = [round(i, 2) for i in box.tolist()]
#         print(f"Detected {model.config.id2label[label.item()]} with confidence {round(score.item(), 3)} at location {box}")

In [8]:
# processor = AutoProcessor.from_pretrained("google/owlv2-base-patch16-ensemble")
# model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble")

# # url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# # image = Image.open(requests.get(url, stream=True).raw)
# # texts = [["a photo of a cat", "a photo of a dog"]]
# # inputs = processor(text=texts, images=image, return_tensors="pt")



# for image_path in data['image']:
#     # Load an image
#     image_path = input_dir / "images" / image_path  # Change this to your image path
#     image = cv2.imread(str(image_path))
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

#     # image = torch.tensor(image)
    
#     # Preprocess the image
#     inputs = processor(texts = ["a photo of an airplane"], images=image, return_tensors="pt")

# # forward pass
# with torch.no_grad():
#     outputs = model(**inputs)

# # Note: boxes need to be visualized on the padded, unnormalized image
# # hence we'll set the target image sizes (height, width) based on that

# def get_preprocessed_image(pixel_values):
#     pixel_values = pixel_values.squeeze().numpy()
#     unnormalized_image = (pixel_values * np.array(OPENAI_CLIP_STD)[:, None, None]) + np.array(OPENAI_CLIP_MEAN)[:, None, None]
#     unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
#     unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
#     unnormalized_image = Image.fromarray(unnormalized_image)
#     return unnormalized_image

# unnormalized_image = get_preprocessed_image(inputs.pixel_values)

# target_sizes = torch.Tensor([unnormalized_image.size[::-1]])
# # Convert outputs (bounding boxes and class logits) to final bounding boxes and scores
# results = processor.post_process_object_detection(
#     outputs=outputs, threshold=0.2, target_sizes=target_sizes
# )

# i = 0  # Retrieve predictions for the first image for the corresponding text queries
# text = texts[i]
# boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

# for box, score, label in zip(boxes, scores, labels):
#     box = [round(i, 2) for i in box.tolist()]
#     print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")


In [9]:
# from transformers import pipeline
# import skimage
# import numpy as np
# from PIL import Image, ImageDraw
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# checkpoint = "google/owlv2-base-patch16-ensemble" #google/owlv2-base-patch16-ensemble
# detector = pipeline(model=checkpoint, task="zero-shot-object-detection")


# for image_path in data['image']:
#     # Load an image
#     image_path = input_dir / "images" / image_path  # Change this to your image path
#     image = cv2.imread(str(image_path))
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image_pil = Image.fromarray(image)

#     predictions = detector(
#         image_pil,
#         candidate_labels=["airplane", "missile", "drone", "helicopter", "fighter jet"],
#         threshold = 0.011
#     )
#     print(f"Predictions for {image_path}:")
#     fig, ax = plt.subplots(1)
#     ax.imshow(image_pil)

#     for prediction in predictions:
#         label = prediction['label']
#         score = prediction['score']
#         box = prediction['box']
#         print(f"Detected {label} with confidence {round(score, 3)} at location {box}")

#         # Draw bounding box
#         rect = patches.Rectangle(
#             (box['xmin'], box['ymin']),
#             box['xmax'] - box['xmin'],
#             box['ymax'] - box['ymin'],
#             linewidth=2,
#             edgecolor='r',
#             facecolor='none'
#         )
#         ax.add_patch(rect)
#         plt.text(
#             box['xmin'], box['ymin'] - 10,
#             f"{label}: {round(score, 3)}",
#             color='red',
#             fontsize=12,
#             bbox=dict(facecolor='white', alpha=0.5)
#         )
    
#     plt.show()



In [8]:
from transformers import pipeline
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

checkpoint = "onnx-community/yolov10s"  # DINOv2 model
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

for image_path in data['image']:
    # Load an image
    image_path = input_dir / "images" / image_path  # Change this to your image path
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image)

    predictions = detector(
        image_pil,
        candidate_labels=["airplane", "missile", "drone", "helicopter", "fighter jet"],
        threshold=0.16
    )
    print(f"Predictions for {image_path}:")
    fig, ax = plt.subplots(1)
    ax.imshow(image_pil)

    for prediction in predictions:
        label = prediction['label']
        score = prediction['score']
        box = prediction['box']
        

        if score > 0.16:  # Check if confidence score is greater than 0.16
            print(f"Detected {label} with confidence {round(score, 3)} at location {box}")
            # Draw bounding box
            rect = patches.Rectangle(
                (box['xmin'], box['ymin']),
                box['xmax'] - box['xmin'],
                box['ymax'] - box['ymin'],
                linewidth=2,
                edgecolor='r',
                facecolor='none'
            )
            ax.add_patch(rect)
            plt.text(
                box['xmin'], box['ymin'] - 10,
                f"{label}: {round(score, 3)}",
                color='red',
                fontsize=12,
                bbox=dict(facecolor='white', alpha=0.5)
            )
    
    plt.show()
# ['GroundingDinoForObjectDetection', 'Owlv2ForObjectDetection', 'OwlViTForObjectDetection']

config.json:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

ValueError: The checkpoint you are trying to load has model type `yolov10` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [11]:
# draw = ImageDraw.Draw(image)

# for prediction in predictions:
#     box = prediction["box"]
#     label = prediction["label"]
#     score = prediction["score"]

#     xmin, ymin, xmax, ymax = box.values()
#     draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
#     draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="white")

# image

# from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

# model = AutoModelForZeroShotObjectDetection.from_pretrained(checkpoint)
# processor = AutoProcessor.from_pretrained(checkpoint)

# import requests

# # image prompts
# url = "https://unsplash.com/photos/oj0zeY2Ltk4/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MTR8fHBpY25pY3xlbnwwfHx8fDE2Nzc0OTE1NDk&force=true&w=640"
# im = Image.open(requests.get(url, stream=True).raw)

# # text prompts
# text_queries = ["hat", "book", "sunglasses", "camera"]
# inputs = processor(text=text_queries, images=im, return_tensors="pt")

# # printing the image
# im

# import torch

# with torch.no_grad():
#     outputs = model(**inputs)
#     target_sizes = torch.tensor([im.size[::-1]])
#     results = processor.post_process_object_detection(outputs, threshold=0.1, target_sizes=target_sizes)[0]

# draw = ImageDraw.Draw(im)

# scores = results["scores"].tolist()
# labels = results["labels"].tolist()
# boxes = results["boxes"].tolist()

# for box, score, label in zip(boxes, scores, labels):
#     xmin, ymin, xmax, ymax = box
#     draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
#     draw.text((xmin, ymin), f"{text_queries[label]}: {round(score,2)}", fill="white")
#     print('Query:'+ text_queries[label], round(score,2))
#     print(f"\t {' '.join(str(i) for i in [xmin, ymin, xmax, ymax])}")



In [10]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
from ultralytics import YOLO

# Initialize CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Initialize YOLOv8 model for object detection
yolo_model = YOLO("yolov9n.pt")  # Load a pre-trained YOLOv8 model

candidate_labels = ["grey camouflage fighter jet"]

# Function to process each image
def process_image(image_path):
    # Load an image
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image)

    # Perform object detection using YOLOv8
    results = yolo_model(image)

    # Initialize plot
    fig, ax = plt.subplots(1)
    ax.imshow(image_pil)

    # Track the highest confidence detection
    highest_confidence = 0
    best_box = None
    best_label = None

    # Process YOLO results
    for result in results:  # Process each result in the results list
        boxes = result.boxes
        names = result.names

        for box in boxes:
            # Extract bounding box coordinates
            xmin, ymin, xmax, ymax = box.xyxy[0].tolist()[:4]
            confidence = box.conf[0].item()
            class_id = int(box.cls[0].item())
            label = names[class_id]

            # Crop the detected object
            cropped_image = image_pil.crop((xmin, ymin, xmax, ymax))

            # Prepare the cropped image and candidate labels for CLIP model
            inputs = clip_processor(text=candidate_labels, images=cropped_image, return_tensors="pt", padding=True)
            outputs = clip_model(**inputs)

            # Get the similarity scores
            logits_per_image = outputs.logits_per_image.softmax(dim=1)
            probs = logits_per_image.cpu().detach().numpy()

            # Find the best matching label
            best_score = probs[0, 0]

            if best_score > highest_confidence:
                highest_confidence = best_score
                best_box = (xmin, ymin, xmax, ymax)
                best_label = candidate_labels[0]

    if best_box and highest_confidence > 0.01:
        xmin, ymin, xmax, ymax = best_box
        print(f"Detected {best_label} with confidence {round(highest_confidence, 3)} at location {(xmin, ymin, xmax, ymax)}")

        # Draw bounding box
        rect = patches.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            linewidth=2,
            edgecolor='r',
            facecolor='none'
        )
        ax.add_patch(rect)
        plt.text(
            xmin, ymin - 10,
            f"{best_label}: {round(highest_confidence, 3)}",
            color='red',
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.5)
        )

    plt.show()

for image_path in data['image']:
    process_image(input_dir / "images" / image_path)


FileNotFoundError: [Errno 2] No such file or directory: 'yolov9n.pt'

In [17]:
for i in results:
    print(i.boxes)
    print(i.names)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([14., 33., 33., 33.], device='cuda:0')
conf: tensor([0.3610, 0.3252, 0.2793, 0.2616], device='cuda:0')
data: tensor([[7.0570e+02, 5.0367e+02, 7.8017e+02, 5.6904e+02, 3.6103e-01, 1.4000e+01],
        [9.1406e+02, 1.6726e+02, 9.6219e+02, 3.1795e+02, 3.2515e-01, 3.3000e+01],
        [5.2833e+02, 1.1714e+02, 6.2643e+02, 1.6150e+02, 2.7928e-01, 3.3000e+01],
        [1.0287e+03, 7.4920e+01, 1.0588e+03, 1.0951e+02, 2.6162e-01, 3.3000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (870, 1520)
shape: torch.Size([4, 6])
xywh: tensor([[ 742.9305,  536.3507,   74.4694,   65.3706],
        [ 938.1257,  242.6085,   48.1311,  150.6889],
        [ 577.3758,  139.3207,   98.1010,   44.3627],
        [1043.7175,   92.2169,   30.0991,   34.5939]], device='cuda:0')
xywhn: tensor([[0.4888, 0.6165, 0.0490, 0.0751],
        [0.6172, 0.2789, 0.0317, 0.1732],
        [0.3799, 0.1601, 0.0645, 0.0510],
        [0.6867, 0.1060, 0.

In [ ]:
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from transformers import OwlV2Config, OwlV2ForObjectDetection

class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.images = sorted(os.listdir(image_dir))
        self.annotations = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        ann_path = os.path.join(self.annotation_dir, self.annotations[idx])
        image = Image.open(img_path).convert("RGB")
        with open(ann_path) as f:
            annotation = json.load(f)
        boxes = torch.tensor(annotation['boxes'], dtype=torch.float32)
        labels = torch.tensor([self.label_to_int(label) for label in annotation['labels']])
        target = {"boxes": boxes, "labels": labels}

        if self.transform:
            image = self.transform(image)

        return image, target

    def label_to_int(self, label):
        # Implement a method to convert label names to integer indices
        pass

# Load the dataset
transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
])
dataset = CustomDataset(image_dir="dataset/images", annotation_dir="dataset/annotations", transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Load the model
config = OwlV2Config()
model = OwlV2ForObjectDetection(config)
